Como salvar y restaurar modelos
===

* *30 min* | Última modificación: Julio 09, 2019

Adaptado de https://www.tensorflow.org/beta/tutorials/keras/save_and_restore_models

## Preparación

In [1]:
from __future__ import absolute_import, division, print_function

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.0.0-beta1'

## Carga de datos

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## Creación del modelo

In [3]:
def create_model():
    
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, 
                           activation=tf.keras.activations.relu, 
                           input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10, 
                           activation=tf.keras.activations.softmax)
    ])
  
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        metrics=['accuracy'])
  
    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Guardado de puntos de control durante el entrenamiento

In [4]:
##
## Path
##
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)


cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, 
    save_weights_only=True,
    verbose=1)

model = create_model()

model.fit(train_images, 
          train_labels,  
          epochs = 10, 
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback]); 

W0709 16:26:19.418592 139873513322304 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 992/1000 [============================>.] - ETA: 0s - loss: 1.1481 - accuracy: 0.6754
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 511us/sample - loss: 1.1417 - accuracy: 0.6780 - val_loss: 0.7217 - val_accuracy: 0.7740
Epoch 2/10
 960/1000 [===========================>..] - ETA: 0s - loss: 0.4218 - accuracy: 0.8865
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 246us/sample - loss: 0.4201 - accuracy: 0.8870 - val_loss: 0.5088 - val_accuracy: 0.8380
Epoch 3/10
 480/1000 [=============>................] - ETA: 0s - loss: 0.3021 - accuracy: 0.9354
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 213us/sample - loss: 0.2739 - accuracy: 0.9300 - val_loss: 0.4818 - val_accuracy: 0.8520
Epoch 4/10
 576/1000 [================>.............] - ETA: 0s - loss: 0.1927 - accuracy: 0.9549
Epoch

In [5]:
## 
## contenido del directorio
##
!ls -l {checkpoint_dir}

total 4784
-rw-r--r-- 1 vagrant vagrant      71 Jul  9 16:26 checkpoint
-rw-r--r-- 1 vagrant vagrant 4886700 Jul  9 16:26 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 vagrant vagrant    1222 Jul  9 16:26 cp.ckpt.index


## Entrenamiento de un nuevo modelo

In [6]:
model = create_model()

loss, acc = model.evaluate(
    test_images, 
    test_labels)

print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 92us/sample - loss: 2.3604 - accuracy: 0.1290
Untrained model, accuracy: 12.90%


## Carga de los pesos anteriores

In [7]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 49us/sample - loss: 0.4212 - accuracy: 0.8710
Restored model, accuracy: 87.10%


## Opciones

In [8]:
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"

checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5) # salva los pesos cada 5 epochs

model = create_model()

model.save_weights(checkpoint_path.format(epoch=0))

model.fit(
    train_images, 
    train_labels,
    epochs = 50, 
    callbacks = [cp_callback],
    validation_data = (test_images,test_labels),
    verbose=0)

W0709 16:26:22.858520 139873513322304 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
W0709 16:26:23.318456 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0709 16:26:23.319385 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0709 16:26:23.320108 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0709 16:26:23.320736 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0709 16:26:23.321236 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0709 16:26:23.322189 139873513322304 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.re


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [9]:
##
## Contenido del directorio
##
! ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [10]:
##
## Carga el último modelo
##
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [11]:
##
## Se crea un nuevo modelo y se cargan los pesos
##
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 89us/sample - loss: 0.4944 - accuracy: 0.8740
Restored model, accuracy: 87.40%


## Guardado manual de los pesos

In [12]:
model.save_weights('./checkpoints/my_checkpoint')

# Carga los pesos
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 92us/sample - loss: 0.4944 - accuracy: 0.8740
Restored model, accuracy: 87.40%


In [13]:
##
## Formato HDF5
##

model = create_model()

model.fit(
    train_images, 
    train_labels, 
    epochs=5)

model.save('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 255us/sample - loss: 1.1655 - accuracy: 0.6620
Epoch 2/5
1000/1000 [==============================] - 0s 114us/sample - loss: 0.4243 - accuracy: 0.8840
Epoch 3/5
1000/1000 [==============================] - 0s 136us/sample - loss: 0.2806 - accuracy: 0.9250
Epoch 4/5
1000/1000 [==============================] - 0s 114us/sample - loss: 0.2256 - accuracy: 0.9400
Epoch 5/5
1000/1000 [==============================] - 0s 106us/sample - loss: 0.1599 - accuracy: 0.9680


In [14]:
##
## Carga del modelo
##
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

W0709 16:26:33.302357 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.iter
W0709 16:26:33.303262 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_1
W0709 16:26:33.303816 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.beta_2
W0709 16:26:33.304501 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.decay
W0709 16:26:33.305703 139873513322304 util.py:244] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0709 16:26:33.306409 139873513322304 util.py:252] A checkpoint was restored (e.g. tf.train.Checkpoint.restore or tf.keras.Model.load_weights) but not all checkpointed values were used. See above for specific issues. Use expect_partial() on the load status object, e.g. tf.train.Checkpoint.restore(...).expect_partial(), to silence these warnings, or use assert_consumed() to make the check explicit. See https://www.tensorflow.org/alpha/guide/checkpoints#

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [15]:
loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 90us/sample - loss: 0.4320 - accuracy: 0.8600
Restored model, accuracy: 86.00%


## saved_model

In [16]:
model = create_model()

model.fit(train_images, train_labels, epochs=5);

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 217us/sample - loss: 1.1088 - accuracy: 0.7020
Epoch 2/5
1000/1000 [==============================] - 0s 91us/sample - loss: 0.4101 - accuracy: 0.8800
Epoch 3/5
1000/1000 [==============================] - 0s 91us/sample - loss: 0.2908 - accuracy: 0.9110
Epoch 4/5
1000/1000 [==============================] - 0s 94us/sample - loss: 0.2177 - accuracy: 0.9380
Epoch 5/5
1000/1000 [==============================] - 0s 87us/sample - loss: 0.1539 - accuracy: 0.9660


In [17]:
import time
saved_model_path = "./saved_models/{}".format(int(time.time()))

tf.keras.experimental.export_saved_model(model, saved_model_path)

saved_model_path

W0709 16:26:35.618914 139873513322304 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
W0709 16:26:35.620182 139873513322304 export_utils.py:182] Export includes no default signature!
W0709 16:26:35.868068 139873513322304 export_utils.py:182] Export includes no default signature!


'./saved_models/1562689594'

In [18]:
!ls saved_models/

1562689594


In [19]:
##
## Recupera el modelo del disco duro
##
new_model = tf.keras.experimental.load_from_saved_model(saved_model_path)
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.predict(test_images).shape

(1000, 10)

In [21]:
new_model.compile(
    optimizer=model.optimizer, 
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

loss, acc = new_model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1000 [==============================] - 0s 84us/sample - loss: 0.4153 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Limpieza del directorio de trabajo

In [25]:
!rm -Rf saved_models/
!rm -Rf checkpoints/
!rm -Rf training_1/
!rm -Rf training_2/
!rm my_model.h5

rm: cannot remove 'my_model.h5': No such file or directory
